# Финальный проект
## на тему: Исследование любимой продукции на основе комментариев с магазинов: Золотое яблоко и Иль Де Боте за последний месяц
Исполнители:
- Галка Полина Руслановна
- Шония Екатерина Георгиевна
- Малютина Анна Николаевна
- Кусаинов Роман

## Парсинг данных
Ссылки на источники:
- [Документация VK API](https://dev.vk.com/ru/reference)
- [Документация API wall.getComments](https://dev.vk.com/ru/method/wall.getComments)
- [Документация API wall.get](https://dev.vk.com/ru/method/wall.get)

Код основан на документации ВКонтакте

In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import pymorphy2

In [2]:
access_token = 'bb01e286bb01e286bb01e2860ab81977f2bbb01bb01e286dd647517a5bcfa4df0bccfcd'
version = '5.131'

In [3]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize_text(text):
    words = text.split()
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmas)

def get_posts_with_keywords(owner_id, keywords, month_thread):
    posts = []
    offset = 0
    count = 100
    pbar = tqdm(desc="Хаваю посты", unit="посты")
    data_thread = datetime(2024, month_thread, 1).timestamp()

    while True:
        url = 'https://api.vk.com/method/wall.get'
        params = {
            'access_token': access_token,
            'v': version,
            'owner_id': owner_id,
            'count': count,
            'offset': offset
        }
        response = requests.get(url, params=params).json()

        if 'response' in response:
            items = response['response']['items']
            items = [item for item in items if item['date'] >= data_thread]
            if not items:
                break
            posts.extend(items)
            pbar.update(len(items))
            if len(items) < count:
                break
            offset += count
        else:
            print("Error:", response)
            break

    pbar.close()
    return posts

def get_all_comments(owner_id, post_id):
    comments = []
    offset = 0
    count = 100

    while True:
        url = 'https://api.vk.com/method/wall.getComments'
        params = {
            'access_token': access_token,
            'v': version,
            'owner_id': owner_id,
            'post_id': post_id,
            'extended': 1,
            'fields': 'from_id, date, text',
            'count': count,
            'offset': offset
        }
        response = requests.get(url, params=params).json()
        time.sleep(0.34)
        if 'response' in response:
            items = response['response']['items']
            comments.extend(items)
            if len(items) < count:
                break
            offset += count
        else:
            print("Error:", response)
            break

    all_comments = [(com['text'], datetime.fromtimestamp(com['date'])) for com in comments]
    return all_comments

In [4]:
month_now = datetime.now().month
month = month_now - 1
owner_id = -44515433
keywords = ['тонер', 'крем', 'уход для лица', 'помада', 'маска']


In [5]:
posts = get_posts_with_keywords(owner_id, keywords, month)

Хаваю посты: 234post [00:04, 58.00post/s]


In [6]:
dfs = {keyword: [] for keyword in keywords}

for post in tqdm(posts, desc="Шарю комментарии по постам..."):
    post_id = post['id']
    post_text = lemmatize_text(post['text'])
    comments = get_all_comments(owner_id, post_id)
    for keyword in keywords:
        lemmatized_keyword = lemmatize_text(keyword)
        if lemmatized_keyword in post_text:
            for comment, date in comments:
                dfs[keyword].append((comment, date))
dfs = {keyword: pd.DataFrame(data, columns=["Comment", "Date"]) for keyword, data in dfs.items()}

# Для демонстрации выводим первые несколько строк каждого DataFrame
for keyword, df in dfs.items():
    print(f"Keyword: {keyword}")
    print(df.head())


Шарю комментарии по постам...:  49%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                        | 115/234 [08:40<08:58,  4.53s/it]


KeyboardInterrupt: 

## Анализ тональности комментариев

Ссылки на источники:
- [Текст ссылки](https://)
- [Текст ссылки](https://)

In [ ]:
%pip install dostoevsky

In [ ]:
%%python3 -m dostoevsky download fasttext-social-network-model

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer)

## Визуализация результатов

Ссылки на источники:
- [Текст ссылки](https://)
- [Текст ссылки](https://)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 6))
product_counts_df = product_counts_df.sort_values(by='Count', ascending=False)
plt.bar(product_counts_df['Product'], product_counts_df['Count'])
plt.xlabel('Products')
plt.ylabel('Count')
plt.title('Frequency of Cosmetic Products Mentions')
plt.xticks(rotation=45)
plt.show()